In [4]:
#Aryan Roy
#CSI Week5 Assisgnment

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

train_labels = train['SalePrice']
train.drop('SalePrice', axis=1, inplace=True)

all_data = pd.concat([train, test], sort=False)

num_cols = all_data.select_dtypes(include=['number']).columns
for col in num_cols:
    all_data[col] = all_data[col].fillna(all_data[col].median())

cat_cols = all_data.select_dtypes(include=['object']).columns
for col in cat_cols:
    all_data[col] = all_data[col].fillna(all_data[col].mode()[0])

ordinal_features = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                    'HeatingQC', 'KitchenQual', 'GarageQual', 'GarageCond',
                    'PoolQC', 'FireplaceQu']

qual_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, np.nan: 0}

for col in ordinal_features:
    all_data[col] = all_data[col].map(qual_mapping)

all_data = pd.get_dummies(all_data)

all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

X_train = all_data[:len(train)]
X_test = all_data[len(train):]

y_train = train_labels

X_train.to_csv("X_train_processed.csv", index=False)
X_test.to_csv("X_test_processed.csv", index=False)
y_train.to_csv("y_train.csv", index=False)

print("Preprocessing complete!")


Preprocessing complete!


In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import cross_val_score

X_train = pd.read_csv("X_train_processed.csv")
X_test = pd.read_csv("X_test_processed.csv")
y_train = pd.read_csv("y_train.csv")

model = LinearRegression()
model.fit(X_train, y_train)

train_preds = model.predict(X_train)
rmse_train = np.sqrt(mean_squared_error(y_train, train_preds))
print("Training RMSE:", rmse_train)

cv_scores = cross_val_score(model, X_train, y_train, scoring="neg_root_mean_squared_error", cv=10)
cv_rmse = -cv_scores.mean()
print("Cross-Validation RMSE (10-fold):", cv_rmse)

test_preds = model.predict(X_test)

print("There are", len(test_preds), "house prices predicted.")
index = int(input("Enter the index (0 to {}) of the house you want to see the predicted price for: ".format(len(test_preds)-1)))
if 0 <= index < len(test_preds):
    print("Predicted price of house at index", index, "is:", test_preds[index])
else:
    print("Invalid index.")


Training RMSE: 22180.89274733687
Cross-Validation RMSE (10-fold): 33643.94463403646
There are 1459 house prices predicted.
Enter the index (0 to 1458) of the house you want to see the predicted price for: 1
Predicted price of house at index 1 is: [164286.2031257]
